In [1]:
import os
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
import tempfile as tmp
import soundfile as sf
from scipy import signal, stats
from scipy.io import wavfile
from typing import Tuple, Union
from sklearn import model_selection, preprocessing
from tensorflow import keras
from tensorflow.keras import layers, optimizers, utils, metrics
from tensorflow_addons import activations as tfa_activations
from keras_nlp import layers as nlp_layers

In [2]:
def trim_wav_file(path: str, st: int = 0, ed: int = 10) -> np.ndarray:
    (sampling_rate, wave_data) = wavfile.read(filename=path)
    st_sample: int = int(st * sampling_rate)
    ed_sample: int = int(ed * sampling_rate)
    wave_data = wave_data[st_sample:ed_sample]
    del st_sample, ed_sample
    return wave_data

def get_signal_list(path: str) -> list[np.ndarray]:
    file_names: list[str] = glob.glob(pathname=f"{path}/*.wav")
    sample_list: list[np.ndarray] = []
    for file_name in file_names:
        sample: np.ndarray = trim_wav_file(path=file_name, st=0, ed=10)
        sample_list.append(sample)
        del sample
    del file_name, file_names
    return sample_list

def combine_signal_list(signal_list: list[np.ndarray], mn_len: int) -> np.ndarray:
    sample_list: list[np.ndarray] = []
    for sgnal in signal_list:
        sample_list.append(sgnal[:mn_len])
    del sgnal
    return np.stack(sample_list, axis=0)

orig_list: list[np.ndarray] = get_signal_list(path="Dataset/Signal")
noise_list: list[np.ndarray] = get_signal_list(path="Dataset/Noise")

mn: int = 1e9
for orig in orig_list:
    mn = min(mn, orig.shape[0])
del orig
for noise in noise_list:
    mn = min(mn, noise.shape[0])
del noise

orig: np.ndarray = combine_signal_list(orig_list, mn)
noise: np.ndarray = combine_signal_list(noise_list, mn)[:, :, 0]
del mn, orig_list, noise_list

C:\Users\Matt\AppData\Local\Temp\ipykernel_26892\2002002543.py:2: WavFileWarning: Chunk (non-data) not understood, skipping it.
  (sampling_rate, wave_data) = wavfile.read(filename=path)


In [3]:
orig = signal.resample_poly(orig, up=1, down=3, axis=1)
noise = signal.resample(noise, num=1898, axis=1)

In [4]:
def gen_x_y_div(orig: np.ndarray, noise: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    x_list: list[np.ndarray] = []
    y_list: list[np.ndarray] = []
    x_avg_list: list[np.ndarray] = []
    x_sd_list: list[np.ndarray] = []
    for i in range(0, orig.shape[0]):
        for j in range(0, noise.shape[0]):
            combi: np.ndarray = orig[i] + noise[j]
            x_list.append(np.stack([combi, noise[j]], axis=1))
            x_avg_list.append(np.stack([np.average(combi, axis=0), np.average(noise[j], axis=0)], axis=0)[np.newaxis, :])
            x_sd_list.append(np.stack([np.std(combi, axis=0), np.std(noise[j], axis=0)], axis=0)[np.newaxis, :])
            y_list.append(orig[i])
            del combi
        del j
    del i
    x: np.ndarray = np.stack(x_list, axis=0)
    y: np.ndarray = np.stack(y_list, axis=0)
    x_avg: np.ndarray = np.stack(x_avg_list, axis=0)
    x_sd: np.ndarray = np.stack(x_sd_list, axis=0)
    return (x, y, x_avg, x_sd)

(x, y_old, x_avg, x_sd) = gen_x_y_div(orig, noise)
x = (x - x_avg) / x_sd
y = (y_old - x_avg[:, :, [0]]) / x_sd[:, :, [0]]
y = y[:, :, np.newaxis]

In [5]:
# (x_train, x_test, y_train, y_test) = model_selection.train_test_split(x, y, test_size=0.2)

In [6]:
custom_objs: dict = {
    "keras_nlp>PositionEmbedding": nlp_layers.PositionEmbedding,
    "keras_nlp>TransformerEncoder": nlp_layers.TransformerEncoder,
    "keras_nlp>TransformerDecoder": nlp_layers.TransformerDecoder,
    "Addons>mish": tfa_activations.mish,
}
model = keras.models.load_model("Model.h5", custom_objects=custom_objs)

In [7]:
y_pred = model.predict(x)
y_pred = y_pred * x_sd[:, :, [0]] + x_avg[:, :, [0]]
rmse = metrics.RootMeanSquaredError(dtype=np.float64)
rmse.update_state(y_old[:, :, np.newaxis], y_pred)
print(rmse.result().numpy())

8/8 [==============================] - 40s 3s/step
2717.949315058769


In [8]:
print(y_pred[:, :, 0])
print(y_old)

[[  58.80050987  -59.16861121   58.80053848 ...   58.80052894
    58.80052894   58.80052894]
 [  59.1090056   -59.0184014    59.1090056  ...   59.1090438
    59.1090438    59.10903425]
 [  60.38994695  -58.85277977   60.38994695 ...   60.38996623
    60.38997587   60.38996623]
 ...
 [-206.31796678 -356.49782204 -206.31793035 ... -206.31796678
  -206.31791821 -206.31791821]
 [-170.16791673 -379.24298919 -170.16786603 ... -170.16791673
  -170.16791673 -170.16786603]
 [   1.68703159 -503.43634803    1.68703159 ...    1.6871541
     1.6871541     1.68703159]]
[[  66.41497  334.75644  798.471   ...   66.55102 -150.31052 -126.06131]
 [  66.41497  334.75644  798.471   ...   66.55102 -150.31052 -126.06131]
 [  66.41497  334.75644  798.471   ...   66.55102 -150.31052 -126.06131]
 ...
 [-180.37022 -327.0981  -415.47083 ... -416.4989  -543.7176  -347.04242]
 [-180.37022 -327.0981  -415.47083 ... -416.4989  -543.7176  -347.04242]
 [-180.37022 -327.0981  -415.47083 ... -416.4989  -543.7176  -347.04